# <center> Sequential MUD estimation for ODE Systems: RLC </center>

___
<font color='blue'><center>**Carlos del-Castillo-Negrete, Rylan Spence**</center></font>
    <br>
    <center>University of Texas at Austin
    <br>
    carlos.delcastillo@utexas.edu, rylan.spence@utexas.edu</center>
---


# RLC Model

The RLC circuit is the electrical circuit consisting of a resistor of resistance $R$, a coil of inductance $L$, a capacitor of capacitance $C$ and a voltage source arranged in series. If the charge
on the capacitor is $Q$ and the current flowing in the circuit is $I$, the voltage across $R, L$ and $C$ are $R I, L \frac{d I}{d t}$ and $\frac{Q}{C}$ respectively. By the Kirchhoff's law that says that the voltage between any two points has to be independent of the path used to travel between the two points,

$$
L I^{\prime}(t)+R I(t)+\frac{1}{C} Q(t)=V(t)
$$

Assuming that $R, L, C$ and $V$ are known, this is still one differential equation in two unknowns, $I$ and $Q$. However the two unknowns are related by $I(t)=\frac{d Q}{d t}(t)$ so that

$$
L Q^{\prime \prime}(t)+R Q^{\prime}(t)+\frac{1}{C} Q(t)=V(t)
$$

or, differentiating with respect to $t$ and then subbing in $\frac{d Q}{d t}(t)=I(t)$,

$$
L I^{\prime \prime}(t)+R I^{\prime}(t)+\frac{1}{C} I(t)=V^{\prime}(t)
$$

This can be linearized into the following ODE system, by letting $x = [I, I^{\prime}]^T$

$$
x_0^{\prime} = x_1 \\
x_1^{\prime} = \frac{1}{L}V^{\prime} - \frac{R}{L}x_1 - \frac{1}{LC}x_0
$$

## Constant Voltage case

If we assume the a constant voltage, then the system becomes:

$$
x_0^{\prime} = x_1 \\
x_1^{\prime} = - \frac{R}{L}x_1 - \frac{1}{LC}x_0
$$

Where the initial condition becomes $x_0 = [I_0, 0]^T$

In [73]:
def rlc_system_constant(
    states: np.array, times: np.array, *parameters: tuple[float, float, float]
) -> np.array:
    """
    Parameters
    ----------
    states : np.ndarray
        An array of the current states of the system
    time : np.ndarray
        simulation time array
    *parameters : tuple[float, float]
        The parameters of the model: beta, gamma
    Returns
    -------
    np.ndarray
        The derivatives of the states with respect to time
    """
    (r,) = parameters

    xdot = [
        states[1],
        -(r / L) * states[1] - states[0] / (L * C),
    ]
    return xdot


def run_rlc(x0, times, parameter_samples) -> None:
    """
    Runs the RLC model for a specified number of drift windows.
    Uses the initial state, the drift windows, the times, the get_parameters and save_output methods of the class
    to integrate the system of ODEs and save the output for each window.

    Parameters
    ----------
    self : object
        The instance of the class
    """
    return odeint(rlc_system_constant, x0, times, args=parameter_samples)

In [74]:
importlib.reload(alg)

<module 'algorithms' from '/Users/rylanspence/Desktop/Git/DC/DC_Sequential/dc_notebooks/ode_notebook/algorithms.py'>

### Response signal characteristics 

We know that the dampening factor:

$$
\zeta = \frac{R}{2}*\sqrt{\frac{C}{L}}
$$

We know that the regimes of behavior for the RL circuit are:

1. $\zeta < 1 \rightarrow $ Underdamped - Decaying oscillations
1. $\zeta > 1 \rightarrow $ Overdamped - Decay with no oscillations
1. $\zeta = 1 \rightarrow $ Critically damped - fastest decay possible without going into oscillation

In [75]:
L = 100  # L - Inductance in henrys
C = 1e-6  # C - Capicatance in Farads
R = 200  # R - Resistance in ohms

# Damping factor
zeta = R / 2 * np.sqrt(C / L)
print(f"Damping factor: {zeta}")

Damping factor: 0.01


#### CASE: 1-Parameter No-Drift (Insufficient Sampling Rate)

#### <center> Select Seed & Run Model <center>

In [76]:
#       Seeds
#      ---------
# initial | Measure | Outcomes
# -----------------------------
# (452763, 648208)   okay
# (751429, 265168)   good
# (503875, 774940)   really bad
# (986134, 379528)   really good

In [77]:
# rlc1_initial_seed, rlc1_measurement_seed

In [78]:
L = 100  # L - Inductance in henrys
C = 1e-6  # C - Capicatance in Farads
R = 200  # R - Resistance in ohms

# Damping factor
zeta = R / 2 * np.sqrt(C / L)
print(f"Damping factor: {zeta}")

Damping factor: 0.01


In [79]:
sample_ts = 0.099
solve_ts = 1e-3

end_time = 1
num_windows = 5
time_window = end_time / num_windows

I_0 = 10  # Initial current in Amps
x_0 = np.array([I_0, 0])

rlc1_true_param = [R]
rlc1_param_shifts = []
rlc1_diff = [
    0.8,
]

rlc1_initial_seed = np.random.randint(0, 10e5)
rlc1_measurement_seed = np.random.randint(0, 10e5)

rlc1_res = alg.sequential_resampling(
    run_rlc,
    x_0,
    rlc1_true_param,  # np.array([0.5]*4),
    num_samples=1000,
    measurement_noise=0.5,
    diff=rlc1_diff,
    solve_ts=solve_ts,
    sample_ts=sample_ts,
    time_window=time_window,
    end_time=end_time,
    param_shifts=rlc1_param_shifts,
    max_nc=1,
    param_mins=[0, 0, 0],
    init_seed=rlc1_initial_seed,
    obs_seed=rlc1_measurement_seed,
)

Initi: Uniform over [[ 40. 360.]]

====== iteration 0 ======
True Param: [200]
Pushing 1000 samples forward through model
1
2
	 ... SEARCH SUMMARY ...
	1: 2/3 points:
		Best(E(r)) = 1.3348781225489632
		Best(Mud) = [201.7303027]
		Mean(E(r)) = 1.3971483058458052, STD(E(r)) = 0.062270183296841974
		MIN(E(r)) = 1.3348781225489632, MAX(E(r))) = 1.4594184891426472

	Best mud estimate nc = 1, num_ts = 2:
	mud point = [201.7303027]
	E(r) = 1.3348781225489632
	min/max = (0.0, 39.77145952926711)
	 Resampling from previous updated distribution

====== iteration 1 ======
True Param: [200]
Pushing 1000 samples forward through model
1
2
	 ... SEARCH SUMMARY ...
	1: 2/3 points:
		Best(E(r)) = 1.3327946425168051
		Best(Mud) = [200.18074338]
		Mean(E(r)) = 1.3631984129543726, STD(E(r)) = 0.030403770437567368
		MIN(E(r)) = 1.3327946425168051, MAX(E(r))) = 1.3936021833919399

	Best mud estimate nc = 1, num_ts = 2:
	mud point = [200.18074338]
	E(r) = 1.3327946425168051
	min/max = (0.0, 64.56009739446016